In [1]:
## Text-to-text translation
from translate import Translator

## Text-to-speech 
from gtts import gTTS
from pydub import AudioSegment

# Danish speech to text
import os
import torch
import librosa
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

ModuleNotFoundError: No module named 'translate'

# Load audio WAV file with Danish speech and transcribe into text

In [18]:
def transcribe_danish_audio(audio_file_path, sampling_rate=16000):
    # load model and tokenizer
    processor = Wav2Vec2Processor.from_pretrained(
        "chcaa/xls-r-300m-danish-nst-cv9")
    model = Wav2Vec2ForCTC.from_pretrained(
        "chcaa/xls-r-300m-danish-nst-cv9")

    wave, sr = librosa.load(audio_file_path, sr=sampling_rate)

    # tokenize
    input_values = processor(wave, return_tensors="pt", padding="longest", sampling_rate=sampling_rate).input_values  # Batch size 1

    # retrieve logits
    logits = model(input_values).logits

    # take argmax and decode
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    return ' '.join(transcription)


audio_file_path = 'Speech-to-text/common_voice_da_27769309.wav'
danish_transcription = transcribe_danish_audio(audio_file_path)
print(danish_transcription)

og det er jo i sidste instans målet


# Load .txt file with danish text, translate to english text using API, save as .txt file

In [19]:
def translate_file(source_file_path, target_file_path='translated_text.txt', source_lang='da', target_lang='en'):
    
    ## Read the text file
    source_text = open(source_file_path, 'r').read().lower()

    ## Define the translator
    file_translator = Translator(from_lang=source_lang, to_lang=target_lang)

    ## Translate the text
    translation = file_translator.translate(source_text)

    ## Save the translation
    with open(target_file_path, 'w') as f:
        f.write(translation)

    return translation

# trans = translate_file(source_file_path='danish_text1.txt')

# Load .txt file with english text, convert to audio using API, save as .wav file

In [20]:
def TTS_file(source_text_file_path, language='en'):
    ## Read translated text file  ## Read the text file
    translated_text = open(source_text_file_path, 'r').read().lower()
    # print(translated_text)

    ## Read aloud translated text using Google Text-to-Speech
    TTS_speech = gTTS(text=translated_text, lang='en', slow=False)
    TTS_speech.save("TTS_speech.mp3")

    ## Convert mp3 to wav, as gTTS only supports mp3
    sound = AudioSegment.from_mp3("TTS_speech.mp3")
    sound.export("TTS_speech.wav", format="wav")
    
    return

# TTS_file(source_text_file_path='translated_text.txt')

# Load .txt file with danish text, translate to english text using API, save as .txt file, then load .txt file with english text, convert to audio using API, save as .wav file

In [3]:
# def translate_TTS_file(src_file_path, tgt_file_path='translated_text.txt', src_lang='da', tgt_lang='en'):
    
#     translation = translate_file(source_file_path=src_file_path, target_file_path=tgt_file_path, source_lang=src_lang, target_lang=tgt_lang)
#     print(translation)
#     TTS_file(source_text_file_path=tgt_file_path, language=tgt_lang)

#     return

# translate_TTS_file(src_file_path='danish_text1.txt')


In [24]:
def translate_TTS_file(source_text, tgt_file_path='TTS_speech.wav', src_lang='da', tgt_lang='en', print_bol=False):

    ## Define the translator
    file_translator = Translator(from_lang=src_lang, to_lang=tgt_lang)

    ## Print source and translated text if specified
    if print_bol:
        print('Source text: ', source_text)
        print('Translated text: ', file_translator.translate(source_text))

    ## Translate the text
    translated_text = file_translator.translate(source_text)

    ## Read aloud translated text using Google Text-to-Speech
    TTS_speech = gTTS(text=translated_text, lang=tgt_lang, slow=False)
    TTS_speech.save('TTS_speech.mp3')

    ## Convert mp3 to wav, as gTTS only supports mp3
    sound = AudioSegment.from_mp3('TTS_speech.mp3')
    sound.export(tgt_file_path, format="wav")
    
    return

translate_TTS_file(source_text=danish_transcription, print_bol=True)

Source text:  og det er jo i sidste instans målet
Translated text:  Surely this must be the ultimate goal.


/work1/s183921/StarGANv2-VC-emotion/Speech-to-text/env/lib/python3.9/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [Errno 2] No such file or directory: 'ffprobe'